# Apriori 알고리즘 유튜브 영상 실습
https://www.youtube.com/watch?v=iRD65OsBWoE&list=PLVsNizTWUw7FmLj3IMECoauQ_-DUbNF0M&index=11&t=0s

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings

In [3]:
warnings.filterwarnings("ignore")

keyword = '손흥민'


## 1. 데이터프레임 만들기 (csv 파일로 만들기)

In [53]:
import tweepy

CONSUMER_KEY = 'EcSE1SxmnUjo2a0mQmZfyLZ4w'
CONSUMER_SECRET = 'BUYFW8ECMwNwE4V6BpyeogOh5dQldbcKahCD8Oqg679XOkVnrM'
ACCESS_TOKEN_KEY = '1307913079900913664-Fd3tREVrgEtib6vneIi68G4FVb5zmg'
ACCESS_TOKEN_SECRET = 'wwj57rvZLOP5axgYbp3Iwni6pfgmWsIakFmigb96GEgI3'

# 개인정보 인증 요청
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

#인증 수행요청
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)


In [ ]:
## 샘플 데이터

testData = [
{'create_dt':'2020-09-18', 'contents':'손흥민 의조선수 국대 선수님들 파팅요!'},
{'create_dt':'2020-09-01', 'contents':'손흥민 만두 먹고 왔다. test contents'},
{'create_dt':'2020-09-02', 'contents':'손흥민은 만두를 싫어 한다.'},
{'create_dt':'2020-09-03', 'contents':'손흥민은 만두를 만드는것을 좋아한다.'},
{'create_dt':'2020-09-04', 'contents':'TS샴푸! 손흥민 광고모델! 손흥!'}
]


columns = ['생성일', '본문']
df = pd.DataFrame(columns=columns)

# for i in range(1, 100):
#     print("Get data", str(i/500*100), "% complete...")
for data0 in testData:
    data_dt = data0['create_dt']
    data_ct = data0['contents']
    row = [data_dt, data_ct]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)
# print("Get data 100 % complete...")

In [54]:
#트위터 API

columns = ['생성일', '본문']
df = pd.DataFrame(columns=columns)

# 1~ 100페이지까지
for i in range(1, 100):
    print("Get data", str(i/500*100), '% complete..')
    tweets = api.search(keyword)
    for tweet in tweets:
        tweet_text = tweet.text
        created = tweet.created_at
        row = [created, tweet_text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)
print('Get data 100% complete..')

Get data 0.2 % complete..
Get data 0.4 % complete..
Get data 0.6 % complete..
Get data 0.8 % complete..
Get data 1.0 % complete..
Get data 1.2 % complete..
Get data 1.4000000000000001 % complete..
Get data 1.6 % complete..
Get data 1.7999999999999998 % complete..
Get data 2.0 % complete..
Get data 2.1999999999999997 % complete..
Get data 2.4 % complete..
Get data 2.6 % complete..
Get data 2.8000000000000003 % complete..
Get data 3.0 % complete..
Get data 3.2 % complete..
Get data 3.4000000000000004 % complete..
Get data 3.5999999999999996 % complete..
Get data 3.8 % complete..
Get data 4.0 % complete..
Get data 4.2 % complete..
Get data 4.3999999999999995 % complete..
Get data 4.6 % complete..
Get data 4.8 % complete..
Get data 5.0 % complete..
Get data 5.2 % complete..
Get data 5.4 % complete..
Get data 5.6000000000000005 % complete..
Get data 5.800000000000001 % complete..
Get data 6.0 % complete..
Get data 6.2 % complete..
Get data 6.4 % complete..
Get data 6.6000000000000005 % comp

In [55]:
# csv파일로 저장
df.to_csv('tweet_temp.csv', index=False)

## 2. 키워드 추출

#### [2-1) 텍스트 데이터 전처리]

In [4]:
# csv파일 불러오기
df = pd.read_csv('tweet_temp4.csv')
df.head()

,생성일,본문
0,2020-09-22 07:05:25,RT @littletwicesis: อัยต้าววววต้าวหมีทั้งสองน่...
1,2020-09-22 07:05:25,RT @iheregap: ก็มาดิวะ !!! https://t.co/b27IfK...
2,2020-09-22 07:05:25,"RT @cuzimafighter7: 👶hi, I’m *name*\n🐰hi *name..."
3,2020-09-22 07:05:25,고양이는 다른 포유류와 달리 단맛을 느끼도록 하는 유전자 일부가 손실돼 있기 때문에...
4,2020-09-22 07:05:25,RT @hanitweet: “남자가 어떻게 주사를 맞아. 나는 자궁이 없어.”\n“...


In [5]:
import re

# 텍스트 정제 함수 : 할글 이외의 문자는 전부 제거
def text_cleaning(text):
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+') #한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    return result

In [6]:
# 'tweet_text' 피처에 이를 적용합니다.
df['ko_text'] = df['본문'].apply(lambda x: text_cleaning(x))
df.head()

,생성일,본문,ko_text
0,2020-09-22 07:05:25,RT @littletwicesis: อัยต้าววววต้าวหมีทั้งสองน่...,
1,2020-09-22 07:05:25,RT @iheregap: ก็มาดิวะ !!! https://t.co/b27IfK...,
2,2020-09-22 07:05:25,"RT @cuzimafighter7: 👶hi, I’m *name*\n🐰hi *name...",
3,2020-09-22 07:05:25,고양이는 다른 포유류와 달리 단맛을 느끼도록 하는 유전자 일부가 손실돼 있기 때문에...,고양이는다른포유류와달리단맛을느끼도록하는유전자일부가손실돼있기때문에단맛을감지할수가없다고...
4,2020-09-22 07:05:25,RT @hanitweet: “남자가 어떻게 주사를 맞아. 나는 자궁이 없어.”\n“...,남자가어떻게주사를맞아나는자궁이없어맞으면나한테효과있어오빠가빨리스탬프세개찍어왔으면좋겠다...


#### [2-2) KoNLPy를 이용한 키워드 추출 (형태소 분석기)]
#### 한국어 약식 불용어 사전 사용 (http://ranks.nl/stopwords/korean)

In [7]:

from konlpy.tag import Okt
from collections import Counter

korean_stopwords_path = './korean_stopwords.txt'
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

def get_nouns(x):
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)

    #한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) > 1]

    #불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]

    return nouns


In [8]:
df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))
print(df.shape)
df.head()

(9126, 4)


,생성일,본문,ko_text,nouns
0,2020-09-22 07:05:25,RT @littletwicesis: อัยต้าววววต้าวหมีทั้งสองน่...,,[]
1,2020-09-22 07:05:25,RT @iheregap: ก็มาดิวะ !!! https://t.co/b27IfK...,,[]
2,2020-09-22 07:05:25,"RT @cuzimafighter7: 👶hi, I’m *name*\n🐰hi *name...",,[]
3,2020-09-22 07:05:25,고양이는 다른 포유류와 달리 단맛을 느끼도록 하는 유전자 일부가 손실돼 있기 때문에...,고양이는다른포유류와달리단맛을느끼도록하는유전자일부가손실돼있기때문에단맛을감지할수가없다고...,"[고양이, 포유류, 달리, 단맛, 유전자, 일부, 손실, 때문, 단맛, 감지, 다지..."
4,2020-09-22 07:05:25,RT @hanitweet: “남자가 어떻게 주사를 맞아. 나는 자궁이 없어.”\n“...,남자가어떻게주사를맞아나는자궁이없어맞으면나한테효과있어오빠가빨리스탬프세개찍어왔으면좋겠다...,"[남자, 주사, 자궁, 효과, 오빠, 리스, 탬프, 자궁경부암, 정보, 제공, 드라..."


## 3. 연관 키워드 추출하기

!pip install apriori apyori

### [연관 분석 연습]

In [9]:
from apyori import apriori

# 1) 트랜잭션 데이터를 생성
transactions = [
    ['손흥민','시소코'],
    ['손흥민','케인'],
    ['손흥민','케인','포체티노']
]

# 2) 연관 분석을 수행합니다.
results = list(apriori(transactions))
for result in results:
    print(result)

RelationRecord(items=frozenset({'손흥민'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])
RelationRecord(items=frozenset({'시소코'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'시소코'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'케인'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'포체티노'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'포체티노'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'손흥민', '시소코'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민', '시소코'}), confidence=0.3333333333333333, lift=1.0), Or

In [10]:
# 3) 연관 분석 필터기능 사용해보기.
# 지지도 0.5, 신뢰도 0.6, 향상도 1.0 이상이면서 (손흥민, 케인) 처헐 규칙의 크기가 2 이하의 규칙을 추출
results = list(apriori(transactions,
                        min_support=0.5,
                        min_confidence=0.6,
                        min_lift=1.0,
                        max_length=2))

for result in results:
    print(result)

RelationRecord(items=frozenset({'손흥민'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])
RelationRecord(items=frozenset({'케인'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'손흥민', '케인'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민', '케인'}), confidence=0.6666666666666666, lift=1.0), OrderedStatistic(items_base=frozenset({'손흥민'}), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0), OrderedStatistic(items_base=frozenset({'케인'}), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])


### [트위터 연관 키워드 분석]

In [11]:
transactions = df['nouns'].tolist()
transactions = [transaction for transaction in transactions if transaction] # 공백 문자열을 방지합니다.and
print(transactions)

[['고양이', '포유류', '달리', '단맛', '유전자', '일부', '손실', '때문', '단맛', '감지', '다지', '도시락', '고양이', '사탕'], ['남자', '주사', '자궁', '효과', '오빠', '리스', '탬프', '자궁경부암', '정보', '제공', '드라마', '친구', '진우', '산부인과', '방문', '박보검', '사혜'], ['컴백', '동시', '에이', '와꽃길', '에이'], ['브라'], ['에이', '에이'], ['브이', '리트', '레저', '메이커', '주신', '캐릭터', '이름', '후보', '리트', '레저', '보석', '도영'], ['루스', '베이', '버그', '평생', '성차별', '이제', '싸움', '이중', '대기'], ['오늘', '국무회의', '형제', '끼리', '라면', '화재', '중화', '인천', '미추홀', '초등학생', '사건', '관련', '문재인', '대통령', '당부', '말씀', '강민석', '대변인', '브리핑'], ['컴백', '콘서트', '이티', '레전드', '무대', '케미', '워크샵', '오늘', '유튜브', '에이'], ['대리', '티켓팅', '선입', '만원', '환불', '불가', '수고', '비제시', '팔로우', '후뎀', '누락', '메인', '트윗', '공지', '확인', '양식', '경력', '후기', '마음', '확인'], ['마미', '노래'], ['서울시', '마포구', '와우산', '층톤앤', '매너', '일간', '서울시'], ['예담', '거울셀카', '중독', '방예담'], ['이범', '예전', '등교', '빨래', '건조대', '양말', '물어', '처음', '반응', '살피', '물어', '천재', '모든'], ['생일', '이벤트', '트위스', '입학식', '컨셉', '일일', '카페', '콜오', '월일', '하츠', '이스트라', '폴라', '생일', '이벤트', '진행', '추첨', '케이크', '선물'], ['

In [33]:

#ress = list(apriori(transactions, min_support=0.1, min_confidence=0.2, min_lift=0, max_length=2))
ress = list(apriori(transactions, min_support=0.001, min_confidence=0.001, min_lift=2, max_length=2))
# ress = list(apriori(transactions, min_support=0.1, min_confidence=0.2, min_lift=5, max_length=2))



#테스트 진행중.

In [45]:

keyword = '코로나'
columns = ['source(조건)/target(결과)', 'support(지지도)', 'confidence(신뢰도)', 'lift(향상도)']
network_dl = pd.DataFrame(columns=columns)



for result in ress :
    # print(items[0], items[1],result.support, result.ordered_statistics[0].confidence, result.ordered_statistics[0].lift)
    if len(result.items) == 2 :
        items = [x for x in result.items]
        row = [items, result.support, result.ordered_statistics[0].confidence, result.ordered_statistics[0].lift]
        series = pd.Series(row, index=network_dl.columns)
        network_dl = network_dl.append(series, ignore_index=True)

#series.append() = [network_dl['소스/타겟'].apply(lambda x : '손흥민' in x)]

# network_dl['테스트'] = network_dl['소스/타겟']
a = network_dl[network_dl['source(조건)/target(결과)'].apply(lambda x : keyword in x)]
# a = network_dl


# network_dl.head()
a.head()

,source(조건)/target(결과),support(지지도),confidence(신뢰도),lift(향상도)
5,"[코로나, 감염]",0.001781,0.692308,37.615385
34,"[거리, 코로나]",0.001187,0.240000,13.040000
115,"[코로나, 국민]",0.001187,0.084507,4.591549
141,"[뉴스, 코로나]",0.001979,0.104167,5.659722
149,"[대응, 코로나]",0.001187,0.500000,27.166667


In [46]:
test_list = []
for tt in a['source(조건)/target(결과)'] :
    for xx in tt :
        if(xx != '손흥민') :
            test_list.append(xx)


# print(test_list)

In [47]:
a['연관키워드 추출'] = a['source(조건)/target(결과)'].apply(lambda x: [y for y in x if y != keyword])


pd.set_option('display.max_rows', 1000000)
a.sort_values(by=['support(지지도)','confidence(신뢰도)', 'lift(향상도)'], axis='index', ascending=False, inplace=True)

a
# a.head()

,source(조건)/target(결과),support(지지도),confidence(신뢰도),lift(향상도),연관키워드 추출
269,"[발생, 코로나]",0.002573,0.590909,32.106061,[발생]
563,"[코로나, 확산]",0.002573,0.139785,50.452381,[확산]
547,"[진자, 코로나]",0.002375,0.705882,38.352941,[진자]
513,"[코로나, 위해]",0.002375,0.144578,7.855422,[위해]
217,"[코로나, 마스크]",0.002177,0.440000,23.906667,[마스크]
141,"[뉴스, 코로나]",0.001979,0.104167,5.659722,[뉴스]
5,"[코로나, 감염]",0.001781,0.692308,37.615385,[감염]
406,"[코로나, 신천지]",0.001583,0.500000,27.166667,[신천지]
331,"[사회, 코로나]",0.001583,0.235294,12.784314,[사회]
520,"[의료, 코로나]",0.001385,0.500000,27.166667,[의료]
